In [ ]:
#| default_exp documentation


In [ ]:
#| export 
#| hide
import nbdev.showdoc as showdoc 
from IPython.display import display, Markdown, Latex

from nbdev.showdoc import show_doc , DocmentTbl
from nbdev.showdoc import *
from fastcore.docments import  *
from fastcore.docscrape import *
from fastcore.utils import *
import inspect, sys
from textwrap import fill

In [ ]:
#| export
#| hide
def table_string(symb):
    """Function to obtain the table from documentation"""
    parse_doc= showdoc.parse_docstring(symb)
    # check_default
    desc= [item[1].desc for item in parse_doc['Parameters'].items() ]
    default_col = [desc_in  for desc_li in  desc  for desc_in in desc_li if desc_in.find('Default') != -1]
    defaul_val= None
    # print(parse_doc['Parameters'])
    if len(default_col)!=0:
        defaul_val= []
        for item in parse_doc['Parameters'].items():
            val_defaut= [i[i.find('=')+1:-1] for i in item[1].desc  if i.find('Default') != -1 ]
            if len(val_defaut) != 0:
                defaul_val.append(val_defaut[0])
            else:
                defaul_val.append(' ')
    var_col = [var[1].name for var in parse_doc['Parameters'].items()]
    type_col = [var[1].type for var in parse_doc['Parameters'].items()]
    desc_col = [var[1].desc for var in parse_doc['Parameters'].items()]
    
    if defaul_val:
        zip(desc_col, defaul_val)
        desc_col = [desc_pa[0][:-1] if desc_pa[1] != ' ' else desc_pa[0] for desc_pa in  zip(desc_col, defaul_val)  ] 
    if defaul_val:
        table_h=   '| |'    + '*Type*'+'|'+'*Default*'+'|'+'*Details*'+'|\n'
        table_h += '|-----|'+ '-----' +'|'+'-----'    +'|'+'-----'    +'|\n'
    else:
        table_h=   '| |'    + '*Type*'+'|'+'Details'+'|'+'\n'
        table_h += '|-----|'+ '-----' +'|'+'-----'  +'|'+'\n'
    table_t =table_h
    # print('Var_col:', var_col)
    renglones=[]
    for num in enumerate(var_col):
        if defaul_val:
            reg_vl='|'+var_col[num[0]]+'|'+type_col[num[0]]+'|'+defaul_val[num[0]]+'|'+''.join(desc_col[num[0]])+'|'+'\n'
        else:
            reg_vl='|'+var_col[num[0]]+'|'+type_col[num[0]]+'|'+''.join(desc_col[num[0]])+'|'+'\n'
            
        renglones.append(reg_vl) 
    
    # print(renglones)
    for reg in renglones:
        table_t += reg
    ##### Return
    return_row =parse_doc['Returns']
    # print(return_row)
    if len(return_row) !=0 :
        if defaul_val:
            return_row_s = '| **Returns** |' +  return_row.type+'| |'  +''.join(return_row.desc)
        else:
            return_row_s = '| **Returns** |' +   return_row.type+'|'+''.join(return_row.desc)
        table_t +=  return_row_s
    
    return table_t

In [ ]:
#| export 
#| hide
def get_signature(sym):
    """Get the string for the signature"""
    sig = showdoc._wrap_sig(f"{showdoc._fmt_sig(showdoc.signature_ex(sym, eval_str=True))}") if showdoc.signature_ex(sym, eval_str=True) else ''
    re_str = sig[0:7]+qual_name(sym)+sig[7:]
    return re_str 

In [ ]:
#| export 
#| hide
def document_alternative(sym, level= 1):
    """Document function """
    title_h  =  '#'*level+ ' '+ qual_name(sym)+'\n' 
    # ####link
    # srt_nvdeb  =showdoc.NbdevLookup().code(patito.clase_fpp)
    # link= showdoc._ext_link(srt_nvdeb, 'source', 'style="float:right; font-size:smaller"') + '\n\n'
    ####
    desc_func = showdoc.docstring(parse_docstring(sym)['Summary'])+'\n'
    signature = get_signature(sym) +'\n'

    return Markdown(title_h +desc_func+signature+'\n' + table_string(sym))

In [ ]:
def foo(var1, var2, *args, long_var_name="hi", only_seldom_used_keyword=0, **kwargs):
    r"""Summarize the function in one line.

    Several sentences providing an extended description. Refer to
    variables using back-ticks, e.g. `var`.

    Parameters
    ----------
    
    var1 : array_like
        Array_like means all those objects -- lists, nested lists, etc. --
        that can be converted to an array.  We can also refer to
        variables like `var1`. (Default = 4)
    var2 : int
        The type above can either refer to an actual Python type
        (e.g. ``int``), or describe the type of the variable in more
        detail, e.g. ``(N,) ndarray`` or ``array_like``.
    *args : iterable
        Other arguments.
    long_var_name : {'hi', 'ho'}, optional
        Choices in brackets, default first when optional.

    Returns
    -------
    type
        Explanation of anonymous return value of type ``type``.
    describe : type
        Explanation of return value named `describe`.
    out : type
        Explanation of `out`.
    type_without_description

    Other Parameters
    ----------------
    only_seldom_used_keyword : int, optional
        Infrequently used parameters can be described under this optional
        section to prevent cluttering the Parameters section.
    **kwargs : dict
        Other infrequently used keyword arguments. Note that all keyword
        arguments appearing after the first parameter specified under the
        Other Parameters section, should also be described under this
        section.

    Raises
    ------
    BadException
        Because you shouldn't have done that.

    See Also
    --------
    numpy.array : Relationship (optional).
    numpy.ndarray : Relationship (optional), which could be fairly long, in
                    which case the line wraps here.
    numpy.dot, numpy.linalg.norm, numpy.eye

    Notes
    -----
    Notes about the implementation algorithm (if needed).

    This can have multiple paragraphs.

    You may include some math:

    .. math:: X(e^{j\omega } ) = x(n)e^{ - j\omega n}

    And even use a Greek symbol like :math:`\omega` inline.

    References
    ----------
    Cite the relevant literature, e.g. [1]_.  You may also cite these
    references in the notes section above.

    .. [1] O. McNoleg, "The integration of GIS, remote sensing,
       expert systems and adaptive co-kriging for environmental habitat
       modelling of the Highland Haggis using object-oriented, fuzzy-logic
       and neural-network techniques," Computers & Geosciences, vol. 22,
       pp. 585-588, 1996.

    Examples
    --------
    These are written in doctest format, and should illustrate how to
    use the function.

    >>> a = [1, 2, 3]
    >>> print([x + 3 for x in a])
    [4, 5, 6]
    >>> print("a\nb")
    a
    b
    """
    # After closing class docstring, there should be one blank line to
    # separate following codes (according to PEP257).
    # But for function, method and module, there should be no blank lines
    # after closing the docstring.
    pass

In [ ]:
class patito:
    
    def clase_fpp():
        """
        Several sentences providing an extended description. Refer to
        variables using back-ticks, e.g. `var`.

        Parameters
        ----------

        var1 : array_like 
            Array_like means all those objects -- lists, nested lists, etc. --
            that can be converted to an array.  We can also refer to
            variables like `var1`.
            (Default = True)
                
        var2 : int
            The type above can either refer to an actual Python type
            (e.g. ``int``), or describe the type of the variable in more
            detail, e.g. ``(N,) ndarray`` or ``array_like``.
        *args : iterable
            Other arguments.
        long_var_name : {'hi', 'ho'}, optional
            Choices in brackets, default first when optional.

        Returns
        -------
        type
            Explanation of anonymous return value of type ``type``.
        describe : type
            Explanation of return value named `describe`.
            """
        print(4)
        return None

In [ ]:
def aluete_fpp():
        """
        Several sentences providing an extended description. Refer to
        variables using back-ticks, e.g. `var`.

        Parameters
        ----------

        var1 : array_like 
            Array_like means all those objects -- lists, nested lists, etc. --
            that can be converted to an array.  We can also refer to
            variables like `var1`.
            (Default = True)
                
        var2 : int
            The type above can either refer to an actual Python type
            (e.g. ``int``), or describe the type of the variable in more
            detail, e.g. ``(N,) ndarray`` or ``array_like``.
        *args : iterable
            Other arguments.
        long_var_name : {'hi', 'ho'}, optional
            Choices in brackets, default first when optional.

        Returns
        -------
        
            """
        print(54)

In [ ]:
document_alternative(patito.clase_fpp)

# patito.clase_fpp
Several sentences providing an extended description. Refer to
variables using back-ticks, e.g. `var`.
>      patito.clase_fpp()

| |*Type*|*Default*|*Details*|
|-----|-----|-----|-----|
|var1|array_like| True|Array_like means all those objects -- lists, nested lists, etc. --that can be converted to an array.  We can also refer tovariables like `var1`.|
|var2|int| |The type above can either refer to an actual Python type(e.g. ``int``), or describe the type of the variable in moredetail, e.g. ``(N,) ndarray`` or ``array_like``.|
|*args|iterable| |Other arguments.|
|long_var_name|{'hi', 'ho'}, optional| |Choices in brackets, default first when optional.|
| **Returns** |type| |Explanation of anonymous return value of type ``type``.

In [ ]:
document_alternative(foo)

/home/miguel/miniconda3/envs/shac/lib/python3.8/site-packages/fastcore/docscrape.py:225: UserWarning: Unknown section Other Parameters
  else: warn(msg)
/home/miguel/miniconda3/envs/shac/lib/python3.8/site-packages/fastcore/docscrape.py:225: UserWarning: Unknown section Raises
  else: warn(msg)
/home/miguel/miniconda3/envs/shac/lib/python3.8/site-packages/fastcore/docscrape.py:225: UserWarning: Unknown section See Also
  else: warn(msg)
/home/miguel/miniconda3/envs/shac/lib/python3.8/site-packages/fastcore/docscrape.py:225: UserWarning: Unknown section Notes
  else: warn(msg)
/home/miguel/miniconda3/envs/shac/lib/python3.8/site-packages/fastcore/docscrape.py:225: UserWarning: Unknown section References
  else: warn(msg)
/home/miguel/miniconda3/envs/shac/lib/python3.8/site-packages/fastcore/docscrape.py:225: UserWarning: Unknown section Examples
  else: warn(msg)


# foo
Summarize the function in one line.
>      foo(var1, var2, *args, long_var_name='hi', only_seldom_used_keyword=0,
>       **kwargs)

| |*Type*|*Default*|*Details*|
|-----|-----|-----|-----|
|var1|array_like| 4|Array_like means all those objects -- lists, nested lists, etc. --that can be converted to an array.  We can also refer to|
|var2|int| |The type above can either refer to an actual Python type(e.g. ``int``), or describe the type of the variable in moredetail, e.g. ``(N,) ndarray`` or ``array_like``.|
|*args|iterable| |Other arguments.|
|long_var_name|{'hi', 'ho'}, optional| |Choices in brackets, default first when optional.|
| **Returns** |type| |Explanation of anonymous return value of type ``type``.

In [ ]:
document_alternative(aluete_fpp)

# aluete_fpp
Several sentences providing an extended description. Refer to
variables using back-ticks, e.g. `var`.
>      aluete_fpp()

| |*Type*|*Default*|*Details*|
|-----|-----|-----|-----|
|var1|array_like| True|Array_like means all those objects -- lists, nested lists, etc. --that can be converted to an array.  We can also refer tovariables like `var1`.|
|var2|int| |The type above can either refer to an actual Python type(e.g. ``int``), or describe the type of the variable in moredetail, e.g. ``(N,) ndarray`` or ``array_like``.|
|*args|iterable| |Other arguments.|
|long_var_name|{'hi', 'ho'}, optional| |Choices in brackets, default first when optional.|
| **Returns** || |

In [ ]:
#| hide
#| export
def string_recursive_tag( string_tag,
                         **kwargs):
    """
    The string labels to add to the node.
    
    Parameters
    ----------
    level_str : str optional
        The level string to use to to separete the level 

    until_level_str : str 
        The level string finish

    node_str : str 
        The node string to use 

    until_node_str : str
        The node string finish

    noise_str : str
        The noise string 

    tree_rec : str
        tree to obtain the final cluster id

    Returns
    --------
    type : tuple
        level and cluster id from the tag using a tree 
    """
    
    level_str = kwargs.get('level_str', '_L_' )
    until_level_str = kwargs.get('until_level_str', '_l_')
    node_str  = kwargs.get('node_str', '_l_')
    until_node_str = kwargs.get('until_node_str', '_c_')
    noise_str= kwargs.get ('noise_str', '_noise')
    tree = kwargs.get( 'tree_rec', None)
    ### Level
    print(5)

In [ ]:
document_alternative(string_recursive_tag)

/home/miguel/miniconda3/envs/shac/lib/python3.8/site-packages/fastcore/docscrape.py:225: UserWarning: potentially wrong underline length... 
Returns 
-------- in 
The string labels to add to the node.
...
  else: warn(msg)


# string_recursive_tag
The string labels to add to the node.
>      string_recursive_tag(string_tag, **kwargs)

| |*Type*|Details|
|-----|-----|-----|
|level_str|str optional|The level string to use to to separete the level |
|until_level_str|str|The level string finish|
|node_str|str|The node string to use |
|until_node_str|str|The node string finish|
|noise_str|str|The noise string |
|tree_rec|str|tree to obtain the final cluster id|
| **Returns** |tuple|level and cluster id from the tag using a tree 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()